In [ ]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "SeaLLMs/SeaLLMs-v3-1.5B", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.641 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SeaLLMs/SeaLLMs-v3-1.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
prompt= "### Question:\n{}\n### Answer:\n{}"

In [ ]:
data= pd.concat([pd.read_json('1k9_rlhf.json').rename(columns= {'answers': 'answer'}), pd.read_json('4k_rlhf.json')])

data= pd.DataFrame({'text': data.apply(lambda x: prompt.format(x['question'], x['answer']) + tokenizer.eos_token, axis= 1)})

In [ ]:
data= Dataset.from_pandas(data)
data= data.train_test_split(test_size=0.15)X

In [ ]:
train_data= data['train']
eval_data= data['test']

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset= eval_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 50,
        num_train_epochs = 5,
        eval_strategy= "epoch",
        save_strategy="epoch",
        load_best_model_at_end=False,
        metric_for_best_model="loss",
        learning_rate = 1.5e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "output_stft",
    ),
)

Map (num_proc=2):   0%|          | 0/5039 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/890 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.641 GB.
3.393 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,039 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 785
 "-____-"     Number of trainable parameters = 18,464,768


Epoch,Training Loss,Validation Loss
0,1.022700,1.078682
2,0.833800,0.878178
4,0.738300,0.846794


In [ ]:
trainer.evaluate()

{'eval_loss': 0.8467943668365479,
 'eval_runtime': 23.549,
 'eval_samples_per_second': 37.794,
 'eval_steps_per_second': 4.756,
 'epoch': 4.984126984126984}

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2154.9515 seconds used for training.
35.92 minutes used for training.
Peak reserved memory = 17.65 GB.
Peak reserved memory for training = 14.257 GB.
Peak reserved memory % of max memory = 74.658 %.
Peak reserved memory for training % of max memory = 60.306 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(trainer.model) # Unsloth has 2x faster inference!
inputs = tokenizer(
[
    prompt.format(
        "Đường cao tốc là đường gì?", # instruction
        ""
    )
], return_tensors = "pt").to("cuda")

outputs = trainer.model.generate(**inputs, max_new_tokens = 256)
tokenizer.batch_decode(outputs)

['### Question:\nĐường cao tốc là đường gì?\n### Answer:\nĐường cao tốc là đường dành cho xe cơ giới, có dải phân cách chia đường cho xe chạy hai chiều riêng biệt; không giao nhau cùng mức với một hoặc các đường khác; được bố trí đầy đủ trang thiết bị phục vụ, bảo đảm giao thông liên tục, an toàn, rút ngắn thời gian hành trình và Speed limit. | Điều 3 | Khoản 13<|endoftext|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
from peft import load_peft_weights, set_peft_model_state_dict

In [ ]:
trainer.model.save_pretrained("sft_model") # Local saving
tokenizer.save_pretrained("sft_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('sft_model/tokenizer_config.json',
 'sft_model/special_tokens_map.json',
 'sft_model/vocab.json',
 'sft_model/merges.txt',
 'sft_model/added_tokens.json',
 'sft_model/tokenizer.json')

In [ ]:
model.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 0.0046, -0.0153,  0.0017,  ...,  0.0133,  0.0251,  0.0131],
                      [ 0.0238, -0.0071,  0.0131,  ...,  0.0206, -0.0037,  0.0133],
                      [-0.0159, -0.0078,  0.0012,  ...,  0.0051, -0.0183,  0.0079],
                      ...,
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034]],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight',
              tensor([[-0.0630, -0.0278, -0.1162,  ...,  0.0332,  0.0074, -0.0410],
                      [-0.0058, -0.0427,  0.0515,  ..., -0.0591, -0.0236,  0.0091],
                      [-0.0415,  0.0194,  0.0532,  ...,  0.0197,  0.0199,  0.0006],
          

In [ ]:
model.load_state_dict(load_peft_weights('sft_model'), strict= False)

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.wei

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
                (ba

In [ ]:
model.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 0.0046, -0.0153,  0.0017,  ...,  0.0133,  0.0251,  0.0131],
                      [ 0.0238, -0.0071,  0.0131,  ...,  0.0206, -0.0037,  0.0133],
                      [-0.0159, -0.0078,  0.0012,  ...,  0.0051, -0.0183,  0.0079],
                      ...,
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
                      [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034]],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight',
              tensor([[-0.0630, -0.0278, -0.1162,  ...,  0.0332,  0.0074, -0.0410],
                      [-0.0058, -0.0427,  0.0515,  ..., -0.0591, -0.0236,  0.0091],
                      [-0.0415,  0.0194,  0.0532,  ...,  0.0197,  0.0199,  0.0006],
          

In [ ]:
load_peft_weights('sft_model')

{'base_model.model.model.layers.0.mlp.down_proj.lora_A.weight': tensor([[ 4.1262e-04, -4.3221e-03,  1.0166e-02,  ...,  3.9760e-04,
          -7.6311e-03, -1.2138e-02],
         [-1.2494e-03, -7.2470e-03, -5.9471e-03,  ..., -5.5801e-03,
           1.6173e-03, -6.4171e-03],
         [-1.0516e-02,  5.9203e-03,  3.5850e-03,  ..., -4.3191e-03,
          -7.6037e-05, -1.0391e-02],
         ...,
         [-2.3089e-03,  6.1093e-03, -5.3193e-03,  ...,  4.5860e-03,
          -6.8056e-03, -8.6253e-03],
         [ 9.4884e-04, -2.4931e-03,  1.0962e-02,  ...,  4.1838e-03,
          -7.5333e-03, -6.4401e-03],
         [ 7.8380e-03,  2.3500e-03, -1.2202e-03,  ...,  2.8487e-03,
          -3.1660e-03,  3.0054e-03]], device='cuda:0'),
 'base_model.model.model.layers.0.mlp.down_proj.lora_B.weight': tensor([[ 2.5138e-03, -2.9939e-03,  1.5747e-03,  ...,  2.0669e-03,
           2.7972e-03, -4.6604e-03],
         [ 1.4121e-03, -3.8602e-03, -5.5671e-03,  ..., -2.1582e-03,
          -2.8029e-04, -5.8225e-03],
 

In [ ]:
set_peft_model_state_dict(model, load_peft_weights('sft_model'))

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.up_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.down_proj.base_layer.weight', 'base_model.model.model.layers.0.input_layernorm.weight', 'base_model.model.model.layers.0.post_attention_layernorm.weight', 'base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.1.self_attn.q